##### **IMAGE GENERATOR**

Genera varias imagenes que son distintas variaciones de una misma imágen.

In [ ]:
from utils import IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, BATCH_SIZE, EPOCHS, DATA_PATH, loadImage, balanceData

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os   # Manipulacion de directorios y archivos del ordenador.
import random

import seaborn as sns

from skimage.io import imread  # Procesamiento y lectura de imágenes.
              
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [ ]:
files = os.listdir(DATA_PATH)  # Lista de los nombres de los archivos en el directorio especificado.
category = [0 if 'normal' in file else 1 for file in files]  # Columna con la categoría de la imagen: 0 celulas normales, 1 celulas atipicas.
df_cito = pd.DataFrame({
            'filename (X)' : files,
            'category (Y)' : category }).sample(frac=1, random_state=14, ignore_index=True) # Aleatorizamos el orden de las filas para minimizar sesgos en la seleccion de datos.

df_cito

In [ ]:
df["category"] = df["category"].replace({0: 'normal', 1: 'atipica'})

print("Categories:", df['category'].unique())
df.head()

In [ ]:
example_df = df.sample(n=1).reset_index(drop=True)

train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    DATA_PATH, 
    x_col='file_name',
    y_col='category',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
)

In [ ]:
plt.figure(figsize=(12, 12))

for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
train_df, validate_df = train_test_split(df.sample(5500),
                                         test_size=500,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print("Shape train", total_train)
print("Shape validation", total_validate)
validate_df.head()

In [ ]:
train_df.head()

In [ ]:
train_df.category.value_counts()

##### **TRAINING** 

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    DATA_PATH, 
    x_col='file_name',
    y_col='category',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE 
)

##### **VALIDATION** 

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    train_df, 
    DATA_PATH, 
    x_col='filename',
    y_col='category',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

##### **FIT MODEL** 

In [ ]:
history = model.fit(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator
)

In [ ]:
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)